In [ ]:
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
import combat_probability as cp
from icepool import d

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [51]:
# Fig. 1: plots the probability of PCs and monsters winning for a range of difficulties

difficulties = list(np.linspace(0.01, 1.0, 21))

pc_win_pct = []
mon_win_pct = []
for diff in difficulties:
    party_hp = 100
    pc_r_win = 3
    pc_dpr_cv = 0.5

    mon_hp = 100
    mon_r_win = pc_r_win/diff
    mon_dpr_cv = 0.5


    party_dpr = mon_hp/pc_r_win
    party_size = 4
    pc_hp = party_hp/party_size
    pc_dpr = party_dpr/party_size

    mon_dpr = party_hp/mon_r_win

    # construct combatants
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='win probability',
            range=[-0.01,1.01],
            tickformat='.1f',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=0.98, y=0.98,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    """fig.add_trace(go.Scatter(
        x=[diff,diff],
        y=[0,1],
        showlegend=False,
        mode='lines',
        line=dict(
            color='black', 
            width=1,
            dash='dash'
        ),
        hoverinfo='none',
    ))"""

    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        #line_color=color,
        #marker_color=color,
        fillcolor=color,
        #hoveron = 'points+fills', # select where hover is active
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.50,
        text=name,
        showarrow=False,
    )

# plot data
fig.add_trace(go.Scatter(
    x=difficulties,
    y=pc_win_pct,
    name=f'PCs',
    mode='lines',
    line_color=COLOR_LIST[0],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=difficulties,
    y=mon_win_pct,
    name=f'NPCs',
    mode='lines',
    line_color=COLOR_LIST[1],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-diff-deadly-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-diff-deadly-small')

In [ ]:
# Fig. 2: plots the probability of PCs and monsters winning for a range of monster CV values
diff = 0.7 # deadly

party_hp = 100
pc_r_win = 3
pc_dpr_cv = 0.5


mon_hp = 100
mon_r_win = pc_r_win/diff
mon_dpr_cvs = np.linspace(0.2, 0.8, 7)


party_dpr = mon_hp/pc_r_win
party_size = 4
pc_hp = party_hp/party_size
pc_dpr = party_dpr/party_size

mon_dpr = party_hp/mon_r_win


# original
pc_win_pct = []
mon_win_pct = []
for mon_dpr_cv in mon_dpr_cvs:
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]
    #print(e.encounter_length(units='rounds', method='approx'))


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='NPC damage sigma / mean',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='win probability',
            range=[0.0,1],
            tickformat='.1f',
            tick0=0, dtick=0.20,
            minor=dict(tick0=0, dtick=0.10),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=pc_win_pct,
    name=f'PCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=mon_win_pct,
    name=f'NPCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-npc-cv-deadly-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-npc-cv-deadly-small')

In [ ]:
# Fig. 3: plots the probability of PCs and monsters winning for a range of PC CV values
diff = 0.7 # deadly

party_hp = 100
pc_r_win = 3
pc_dpr_cvs = np.linspace(0.2, 0.8, 7)


mon_hp = 100
mon_r_win = pc_r_win/diff
mon_dpr_cv = 0.5


party_dpr = mon_hp/pc_r_win
party_size = 4
pc_hp = party_hp/party_size
pc_dpr = party_dpr/party_size

mon_dpr = party_hp/mon_r_win

# original
pc_win_pct = []
mon_win_pct = []

for pc_dpr_cv in pc_dpr_cvs:
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]
    #print(e.encounter_length(units='rounds', method='approx'))


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='PC damage sigma / mean',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='win probability',
            range=[0,1],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=pc_win_pct,
    name=f'PCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'PC CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=mon_win_pct,
    name=f'NPCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'PC CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-pc-cv-deadly-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-pc-cv-deadly-small')

In [48]:
# plots the probability of PCs and monsters winning for a range encounter lengths
diff = 0.7 # deadly

# original
pc_win_pct = []
mon_win_pct = []
party_r_wins = np.linspace(3, 6, 4)

for party_r_win in party_r_wins:
    party_hp = 100
    #party_r_win = 3
    pc_dpr_cv = 0.5

    mon_hp = 100
    mon_r_win = party_r_win/diff
    mon_dpr_cv = 0.5


    party_dpr = mon_hp/party_r_win
    party_size = 4
    pc_hp = party_hp/party_size
    pc_dpr = party_dpr/party_size

    mon_dpr = party_hp/mon_r_win

    # construct combatants
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]
    #print(e.encounter_length(units='rounds', method='approx'))


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='average encounter length (rounds)',
            range=[2, 7],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='win probability',
            range=[0,1],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=party_r_wins,
    y=pc_win_pct,
    name=f'PCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>PCs</b><br>'+
        'PC CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=party_r_wins,
    y=mon_win_pct,
    name=f'NPCs',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'PC CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-pc-cv-deadly-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-pc-cv-deadly-small')

In [ ]:
# Fig. 4: plot DPR CV for published monsters
import numpy as np
import pandas as pd

def baseline_monster(cr):
    return {
        'CR': cr,
        'hit_points': np.round(16*(cr + 1) if cr < 20 else 368 + 48*(cr - 20), 0),
        'damage_per_round': np.round(6*(cr + 1) if cr < 20 else 132 + 12*(cr - 20), 0),
        'attack_bonus': np.round(3.5 + cr/2, 0),
        'save_dc': np.round(11.5 + cr/2, 0),
        'armor_class': np.round(13.0 + cr/3, 0),
        'save_bonus': np.round(0.0 + cr/2, 0),
    }

def monster_dpr_is_valid(monster):
    dpr = 0.65*baseline_monster(monster['CR'])['damage_per_round']
    return monster['dpr.mean']/dpr > 0.5

dM = pd.read_csv('./monster-variability.csv')
dM['dpr.mean'] = dM['damage.mean']/3
dM['dpr.sigma'] = dM['damage.sigma']/np.sqrt(3)
dM['dpr.cv'] = dM['dpr.sigma']/dM['dpr.mean']
dM['valid'] = dM.apply(lambda r: monster_dpr_is_valid(r), axis=1)
dM = dM[dM['valid'] & dM['CR'].between(0.01,30)]

dM = dM[['CR','dpr.cv']].groupby('CR').mean().reset_index()
dM.sort_values(by='CR', inplace=True)

fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='challenge rating',
            range=[0, 27],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='NPC damage sigma / mean',
            range=[-0.01, 1.01],
            tickformat='.1f',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.05),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=0.99, y=0.99,
            orientation='v',
            tracegroupgap=0,
        ),
    )
)

fig.add_trace(go.Scatter(
    x=dM['CR'], 
    y=dM['dpr.cv'],
    name='published monsters',
    mode='markers',
    hovertemplate=
            'Challenge Rating %{x:.0f}<br>'+
            'DPR CV %{y:.2f}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-monster-dpr-cv-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-monster-dpr-cv-small')

In [ ]:
# Fig. 5: plots the probability of PCs losing over a range of difficulties for each tier of play

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='probability of PCs losing',
            #range=[-0.01,1.01],
            range=[-0.01,0.51],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.01, y=0.03,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        #line_color=color,
        #marker_color=color,
        fillcolor=color,
        #hoveron = 'points+fills', # select where hover is active
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.47,
        text=name,
        showarrow=False,
    )

tiers = [
    {
        'name': 'tier 1',
        'cv': 0.8,
        'dash': 'solid',
    },
    {
        'name': 'tier 2',
        'cv': 0.6,
        'dash': 'dash',
    },
    {
        'name': 'tier 3',
        'cv': 0.4,
        'dash': 'dashdot',
    },
    {
        'name': 'tier 4',
        'cv': 0.2,
        'dash': 'dashdot',
    },
]

colors = iter(COLOR_LIST)
difficulties = list(np.linspace(0.05, 1.0, 40))
for tier in tiers:
    pc_win_pct = []
    mon_win_pct = []
    for diff in difficulties:
        party_hp = 100
        pc_r_win = 3
        pc_dpr_cv = tier['cv']

        mon_hp = 100
        mon_r_win = pc_r_win/diff
        mon_dpr_cv = tier['cv']


        party_dpr = mon_hp/pc_r_win
        party_size = 4
        pc_hp = party_hp/party_size
        pc_dpr = party_dpr/party_size

        mon_dpr = party_hp/mon_r_win

        # construct combatants
        combatants = [
            cp.Combatant(
                group='NPCs',
                name='monster',
                opponent='PCs',
                level=5,
                hit_points = mon_hp,
                damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 1',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 2',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 3',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 4',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
        ]
        e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
        pc_win_pct += [e.group_win_pct('PCs', method='approx')]
        mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

    # plot data
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=mon_win_pct,
        name=tier['name'],
        #name=f'CV = {mon_dpr_cv:.2f} - NPCs',
        mode='lines',
        line_color=next(colors),
        #line_color=COLOR_LIST[1],
        #line_dash=tier['dash'],
        hovertemplate=
            #f'<b>CV = {mon_dpr_cv:.2f} - NPCs</b><br>'+
            f'<b>' + tier['name'] + '</b><br>'+
            'difficulty %{x:.2f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-diff-tiers-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-diff-tiers-small')

In [ ]:
# Fig 6: plots the average damage taken by the PCs and monsters for a range of difficulties
INCLUDE_RANGES = True

pc_win_pct = [1]
pc_hp_loss = [0]
pc_hp_loss_sigma = [0]
pc_hp_loss_ql = [0]
pc_hp_loss_qu = [0]
mon_win_pct = [0]
mon_hp_loss = [1]
mon_hp_loss_sigma = [0]
mon_hp_loss_ql = [1]
mon_hp_loss_qu = [1]

difficulties = list(np.linspace(0.0, 1.0, 21))
for diff in difficulties[1:]:
    #print(diff)
    party_hp = 100
    pc_r_win = 3
    pc_dpr_cv = 0.5

    mon_hp = 100
    mon_r_win = pc_r_win/diff
    mon_dpr_cv = 0.5


    party_dpr = mon_hp/pc_r_win
    party_size = 4
    pc_hp = party_hp/party_size
    pc_dpr = party_dpr/party_size

    mon_dpr = party_hp/mon_r_win

    
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)

    """damage = e.group_damage_distribution('NPCs', method='approx', clip=True)
    pc_hp_loss += [damage.mean()/(party_size*pc_hp)]
    pc_hp_loss_sigma += [damage.sd()/(party_size*pc_hp)]
    pc_hp_loss_ql += [damage.quantile(16)/(party_size*pc_hp)]
    pc_hp_loss_qu += [damage.quantile(84)/(party_size*pc_hp)]

    damage = e.group_damage_distribution('PCs', method='approx', clip=True)
    mon_hp_loss += [damage.mean()/(mon_hp)]
    mon_hp_loss_sigma += [damage.sd()/(mon_hp)]
    mon_hp_loss_ql += [damage.quantile(16)/(mon_hp)]
    mon_hp_loss_qu += [damage.quantile(84)/(mon_hp)]
    
    print(diff)"""

    pc_hp_loss += [e.group_damage_mean('NPCs', method='approx', clip=True)/(party_size*pc_hp)]
    pc_hp_loss_sigma += [e.group_damage_sigma('NPCs', method='approx', clip=True)/(party_size*pc_hp)]
    pc_hp_loss_ql += [pc_hp_loss[-1] - pc_hp_loss_sigma[-1]]
    pc_hp_loss_qu += [pc_hp_loss[-1] + pc_hp_loss_sigma[-1]]

    mon_hp_loss += [e.group_damage_mean('PCs', method='approx', clip=True)/(mon_hp)]
    mon_hp_loss_sigma += [e.group_damage_sigma('PCs', method='approx', clip=True)/(mon_hp)]
    mon_hp_loss_ql += [mon_hp_loss[-1] - mon_hp_loss_sigma[-1]]
    mon_hp_loss_qu += [mon_hp_loss[-1] + mon_hp_loss_sigma[-1]]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='damage taken / hit points',
            range=[-0.01,1.01],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.02, y=0.98,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        fillcolor=color,
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.5,
        text=name,
        showarrow=False,
    )

# plot PC data
if INCLUDE_RANGES:
    y_min = [pc_hp_loss_ql[i] for i in range(len(difficulties))]
    y_max = [pc_hp_loss_qu[i] for i in range(len(difficulties))]
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=y_min,
        name='fill',
        legendgroup='PCs',
        showlegend=False,
        line_color=COLOR_LIST[0],
        line_width=0,
        fill='none',
        hoverinfo='none',
    ))
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=y_max,
        name='fill',
        legendgroup='PCs',
        showlegend=False,
        line_color=COLOR_LIST[0],
        line_width=0,
        fill='tonexty',
        hoverinfo='none',
    ))
fig.add_trace(go.Scatter(
    x=difficulties,
    y=pc_hp_loss,
    name=f'PCs',
    legendgroup='PCs',
    showlegend=True,
    line_color=COLOR_LIST[0],
    fill='none',
    customdata=pc_hp_loss_sigma,
    hovertemplate=
        f'<b>PCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'damage mean %{y:.1%}<br>'+
        'damage sigma %{customdata:.1%}<extra></extra>',
))


# plot NPC data
if INCLUDE_RANGES:
    y_min = [mon_hp_loss_ql[i] for i in range(len(difficulties))]
    y_max = [mon_hp_loss_qu[i] for i in range(len(difficulties))]
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=y_min,
        name='fill',
        legendgroup='NPCs',
        showlegend=False,
        line_color=COLOR_LIST[1],
        line_width=0,
        fill='none',
        hoverinfo='none',
    ))
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=y_max,
        name='fill',
        legendgroup='NPCs',
        showlegend=False,
        line_color=COLOR_LIST[1],
        line_width=0,
        fill='tonexty',
        hoverinfo='none',
    ))
fig.add_trace(go.Scatter(
    x=difficulties,
    y=mon_hp_loss,
    name=f'NPCs',
    legendgroup='NPCs',
    line_color=COLOR_LIST[1],
    fill='none',
    customdata=mon_hp_loss_sigma,
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'damage mean %{y:.1%}<br>'+
        'damage sigma %{customdata:.1%}<extra></extra>',
))


fig.add_trace(go.Scatter(
    x=[0,1],
    y=[0,1],
    name=f'1-to-1',
    showlegend=False,
    mode='lines',
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-damage-vs-diff-large', selector={'name': 'none'})
    tfb.save_fig_html(fig, format='small', name=f'./fig-damage-vs-diff-small', selector={'name': 'none'})

In [ ]:
# Fig 7: plots the probability of an encounter ending up in each difficulty range
# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), 
    ('TPK',     1.00, 0.10, 'rgba(144,  79, 213, 0.15)'), 
]

difficulties = [
    ('Easy',   0.5*(0.15 + 0.30)), 
    ('Medium', 0.5*(0.30 + 0.45)), 
    ('Hard',   0.5*(0.45 + 0.70)), 
    ('Deadly', 0.5*(0.70 + 1.00)),
]
"""difficulties = [
    ('Easy',   1.1*(0.15)), 
    ('Medium', 1.1*(0.30)), 
    ('Hard',   1.1*(0.45)), 
    ('Deadly', 1.1*(0.70)),
]"""

results = []
for name, diff in difficulties:
    #diff = encounter_difficulty[1] + encounter_difficulty[2]/2
    party_hp = 100
    pc_r_win = 3
    pc_dpr_cv = 0.5

    mon_hp = 100
    mon_r_win = pc_r_win/diff
    mon_dpr_cv = 0.5


    party_dpr = mon_hp/pc_r_win
    party_size = 4
    pc_hp = party_hp/party_size
    pc_dpr = party_dpr/party_size

    mon_dpr = party_hp/mon_r_win

    # construct combatants
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)

    pc_hp_loss = e.group_damage_distribution('NPCs', method='approx', clip=True)
    probs = []
    for ed in thresholds:
        x0 = int(np.round(ed[1]*party_hp))
        x1 = int(np.round((ed[1]+ed[2])*party_hp))
        probs += [sum([pc_hp_loss.probability(x) for x in range(x0,x1)])]
    results.append(probs)
    #print(probs, sum(probs))


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
        ),
        yaxis=dict(
            title_text='probability',
            range=[0,1],
            tickformat='.0%',
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=0.99, y=0.99,
            orientation='h',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

results = np.transpose(results)
#x = [t[0] for t in thresholds]
x = [d[0] for d in difficulties]

for t, p, in zip(thresholds, results):
    fig.add_trace(go.Bar(
        name=t[0], 
        x=x, 
        y=p, 
        showlegend=True,
        hovertemplate=
            'target difficulty %{x}<br>'+
            'final difficult '+t[0]+'<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

# Change the bar mode
fig.update_layout(barmode='group')

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-diff-probabilities-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-diff-probabilities-small')

In [ ]:
# plots the average damage taken by the PCs and monsters for a range of difficulties

difficulties = list(np.linspace(0.0, 1.0, 21))
rounds_mean = [3.12]
rounds_sigma = [0.44]
for diff in difficulties[1:]:
    party_hp = 100
    pc_r_win = 3
    pc_dpr_cv = 0.5

    mon_hp = 100
    mon_r_win = pc_r_win/diff
    mon_dpr_cv = 0.5

    party_dpr = mon_hp/pc_r_win
    party_size = 4
    pc_hp = party_hp/party_size
    pc_dpr = party_dpr/party_size

    mon_dpr = party_hp/mon_r_win
    
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
    rounds = e.encounter_length_distribution(units='rounds', method='approx', clip=True)
    rounds_mean += [rounds.mean()]
    rounds_sigma += [rounds.sd()]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='encounter length (rounds)',
            range=[0,4],
            #tickformat='.0%'
            tick0=0, dtick=1.0,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.02, y=0.98,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[4.0, 4.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        fillcolor=color,
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=2.0,
        text=name,
        showarrow=False,
    )

# plot PC data
y_min = [rounds_mean[i] - rounds_sigma[i] for i in range(len(difficulties))]
y_max = [rounds_mean[i] + rounds_sigma[i] for i in range(len(difficulties))]
fig.add_trace(go.Scatter(
    x=difficulties,
    y=y_min,
    legendgroup='PCs',
    showlegend=False,
    line_color=COLOR_LIST[0],
    line_width=0,
    fill='none',
    hoverinfo='none',
))
fig.add_trace(go.Scatter(
    x=difficulties,
    y=y_max,
    legendgroup='PCs',
    showlegend=False,
    line_color=COLOR_LIST[0],
    line_width=0,
    fill='tonexty',
    hoverinfo='none',
))
fig.add_trace(go.Scatter(
    x=difficulties,
    y=rounds_mean,
    name=f'PCs',
    legendgroup='PCs',
    showlegend=False,
    line_color=COLOR_LIST[0],
    fill='none',
    customdata=rounds_sigma,
    hovertemplate=
        f'<b>PCs</b><br>'+
        'difficulty %{x:.2f}<br>'+
        'rounds mean %{y:.2f}<br>'+
        'rounds sigma %{customdata:.2f}<extra></extra>',
))




"""fig.add_trace(go.Scatter(
    x=[0,1],
    y=[0,1],
    name=f'1-to-1',
    showlegend=False,
    mode='lines',
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))"""

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-damage-vs-diff-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-damage-vs-diff-small')

In [ ]:
# plots the average damage taken relative to max HP for a range of monster CoV values

diff = 0.2 # deadly

party_hp = 100
party_dpr = 40
party_size = 4
pc_hp = party_hp/4
pc_dpr = party_dpr/4
pc_dpr_cv = 0.5

mon_hp = 100
mon_dpr = (party_hp*diff)/(mon_hp/party_dpr)

# original
pc_win_pct = []
mon_win_pct = []
pc_hp_loss = []
pc_hp_loss_sigma = []
mon_hp_loss = []
mon_hp_loss_sigma = []
mon_dpr_cvs = np.linspace(0.2, 0.8, 7)
for mon_dpr_cv in mon_dpr_cvs:
    #print(f'{mon_dpr_cv:.1f}')
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

    pc_hp_loss += [e.group_damage_mean('NPCs', method='approx', clip=True)/(party_size*pc_hp)]
    pc_hp_loss_sigma += [e.group_damage_sigma('NPCs', method='approx', clip=True)/(party_size*pc_hp)]

    mon_hp_loss += [e.group_damage_mean('PCs', method='approx', clip=True)/(mon_hp)]
    mon_hp_loss_sigma += [e.group_damage_sigma('PCs', method='approx', clip=True)/(mon_hp)]



# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='DPR CV',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='damage / hit points',
            range=[0,1.02],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=pc_hp_loss,
    name=f'PCs',
    #line_color=COLOR_LIST[i],
    error_y=dict(
        type='data',
        symmetric=False,
        array=pc_hp_loss_sigma,
        arrayminus=pc_hp_loss_sigma,
    ),
    hovertemplate=
        f'<b>PCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=mon_hp_loss,
    name=f'NPCs',
    #line_color=COLOR_LIST[i],
    #error_y=dict(
    #    type='data',
    #    symmetric=True,
    #    array=mon_hp_loss_sigma,
    #),
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=np.array(pc_hp_loss_sigma)/np.array(pc_hp_loss),
    name=f'sigma/mean',
    #line_color=COLOR_LIST[i],
    #error_y=dict(
    #    type='data',
    #    symmetric=True,
    #    array=mon_hp_loss_sigma,
    #),
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-example-dt-distribution-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-example-dt-distribution-small')

In [5]:
# plots the average damage taken by the PCs and monsters for a range of difficulties

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='damage taken / hit points',
            range=[-0.01,1.01],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.01, y=0.99,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

# plot difficulty ranges
thresholds = [
    ('Trivial', 0.00, 0.15, 'rgba(167, 167, 167, 0.15)'),
    ('Easy',    0.15, 0.15, 'rgba(  0, 199, 151, 0.15)'),
    ('Medium',  0.30, 0.15, 'rgba(245, 166,  35, 0.15)'),
    ('Hard',    0.45, 0.25, 'rgba(228,  90,  29, 0.15)'),
    ('Deadly',  0.70, 0.30, 'rgba(144,  79, 213, 0.15)'), #rgba(213,  79,  79, 0.2)'
]
for name, diff, dx, color in thresholds:
    fig.add_trace(go.Scatter(
        x=[diff, diff+dx], 
        y=[1.0, 1.0],
        mode='none',
        fill='tozeroy', 
        showlegend=False,
        #line_color=color,
        #marker_color=color,
        fillcolor=color,
        #hoveron = 'points+fills', # select where hover is active
        hoverinfo='none'
    ))
    fig.add_annotation(
        x=diff + dx/2, 
        y=0.5,
        text=name,
        showarrow=False,
    )

tiers = [
    {
        'name': 'tier 1',
        'cv': 0.8,
        'dash': 'solid',
    },
    {
        'name': 'tier 2',
        'cv': 0.6,
        'dash': 'dash',
    },
    {
        'name': 'tier 3',
        'cv': 0.4,
        'dash': 'dashdot',
    },
    {
        'name': 'tier 4',
        'cv': 0.2,
        'dash': 'dashdot',
    },
]

colors = iter(COLOR_LIST)
difficulties = list(np.linspace(0.05, 1.0, 40))
for tier in tiers:
    pc_win_pct = []
    mon_win_pct = []
    pc_hp_loss = []
    pc_hp_loss_sigma = []
    mon_hp_loss = []
    mon_hp_loss_sigma = []
    for diff in difficulties:
        party_hp = 100
        pc_r_win = 3
        pc_dpr_cv = tier['cv']

        mon_hp = 100
        mon_r_win = pc_r_win/diff
        mon_dpr_cv = tier['cv']


        party_dpr = mon_hp/pc_r_win
        party_size = 4
        pc_hp = party_hp/party_size
        pc_dpr = party_dpr/party_size

        mon_dpr = party_hp/mon_r_win

        
        combatants = [
            cp.Combatant(
                group='NPCs',
                name='monster',
                opponent='PCs',
                level=5,
                hit_points = mon_hp,
                damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 1',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 2',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 3',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
            cp.Combatant(
                group='PCs',
                name='PC 4',
                opponent='NPCs',
                level=5,
                hit_points = pc_hp,
                damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
            ),
        ]
        e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=10)
        pc_win_pct += [e.group_win_pct('PCs', method='approx')]
        mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

        pc_hp_loss += [e.group_damage_mean('NPCs', method='approx', clip=False)/(party_size*pc_hp)]
        #pc_hp_loss_sigma += [e.group_damage_sigma('NPCs', method='approx', clip=True)/(party_size*pc_hp)]

        #mon_hp_loss += [e.group_damage_mean('PCs', method='approx', clip=True)/(mon_hp)]
        #mon_hp_loss_sigma += [e.group_damage_sigma('PCs', method='approx', clip=True)/(mon_hp)]


    # plot data
    fig.add_trace(go.Scatter(
        x=difficulties,
        y=pc_hp_loss,
        name=tier['name'],
        mode='lines',
        line_color=next(colors),
        hovertemplate=
            f'<b>' + tier['name'] + '</b><br>'+
            'difficulty %{x:.2f}<br>'+
            'HP lost %{y:.1%}<extra></extra>',
    ))

    """fig.add_trace(go.Scatter(
        x=difficulties,
        y=mon_hp_loss,
        name=f'NPCs',
        mode='lines',
        line_color=COLOR_LIST[1],
        hovertemplate=
            f'<b>NPCs</b><br>'+
            'difficulty %{x:.2f}<br>'+
            'HP lost %{y:.1%}<extra></extra>',
    ))"""

fig.add_trace(go.Scatter(
    x=[0,1],
    y=[0,1],
    name=f'1-to-1',
    showlegend=False,
    mode='lines',
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-win-vs-diff-deadly-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-win-vs-diff-deadly-small')

In [4]:
# plots the average damage taken relative to max HP for a range of monster CoV values


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='DPR CV',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='damage CV',
            range=[0,1.02],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# encounter parameters
diff = 0.2 # deadly

party_hp = 100
party_r_win = 3

mon_hp = 100

party_dpr = mon_hp/pc_r_win
party_size = 4
pc_hp = party_hp/party_size
pc_dpr = party_dpr/party_size
pc_dpr_cv = 0.5

mon_r_win = pc_r_win/diff
mon_dpr = party_hp/mon_r_win

# original
pc_win_pct = []
mon_win_pct = []
pc_hp_loss = []
pc_hp_loss_sigma = []
mon_hp_loss = []
mon_hp_loss_sigma = []
mon_dpr_cvs = np.linspace(0.2, 0.8, 7)
for mon_dpr_cv in mon_dpr_cvs:
    #print(f'{mon_dpr_cv:.1f}')
    combatants = [
        cp.Combatant(
            group='NPCs',
            name='monster',
            opponent='PCs',
            level=5,
            hit_points = mon_hp,
            damage = [cp.gaussian_dice_distribution(mon_dpr, mon_dpr_cv*mon_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 1',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 2',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 3',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
        cp.Combatant(
            group='PCs',
            name='PC 4',
            opponent='NPCs',
            level=5,
            hit_points = pc_hp,
            damage = [cp.gaussian_dice_distribution(pc_dpr, pc_dpr_cv*pc_dpr)],
        ),
    ]
    e = cp.Encounter(combatants, initiative=['PC 1','PC 2','monster','PC 3','PC 4'], rounds=8)
    pc_win_pct += [e.group_win_pct('PCs', method='approx')]
    mon_win_pct += [e.group_win_pct('NPCs', method='approx')]

    pc_hp_loss += [e.group_damage_mean('NPCs', method='approx', clip=True)/(party_size*pc_hp)]
    pc_hp_loss_sigma += [e.group_damage_sigma('NPCs', method='approx', clip=True)/(party_size*pc_hp)]

    mon_hp_loss += [e.group_damage_mean('PCs', method='approx', clip=True)/(mon_hp)]
    mon_hp_loss_sigma += [e.group_damage_sigma('PCs', method='approx', clip=True)/(mon_hp)]


fig.add_trace(go.Scatter(
    x=mon_dpr_cvs,
    y=np.array(pc_hp_loss_sigma)/np.array(pc_hp_loss),
    name=f'sigma/mean',
    #line_color=COLOR_LIST[i],
    #error_y=dict(
    #    type='data',
    #    symmetric=True,
    #    array=mon_hp_loss_sigma,
    #),
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

x = np.linspace(0.05, 0.95, 20)
y = x/np.sqrt(party_r_win)
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    name=f'theory',
    #line_color=COLOR_LIST[i],
    #error_y=dict(
    #    type='data',
    #    symmetric=True,
    #    array=mon_hp_loss_sigma,
    #),
    hovertemplate=
        f'<b>NPCs</b><br>'+
        'monster CV %{x:.2f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-example-dt-distribution-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-example-dt-distribution-small')